In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
### The below is an example of how a RAG prompt may look:
"""
Answer the question based on the context below,                 }
if you cannot answer the question using the                     }--->  (Rules) For Our Prompt
provided information answer with "I don't know"                 }

Context: Aurelio AI is an AI development studio                 }
focused on the fields of Natural Language Processing (NLP)      }
and information retrieval using modern tooling                  }--->   Context AI has
such as Large Language Models (LLMs),                           }
vector databases, and LangChain.                                }

Question: Does Aurelio AI do anything related to LangChain?     }--->   User Question

Answer:                                                         }--->   AI Answer
Here we can see how the AI will appoach our question, as you can see we have a formulated response, if the context has the answer, then use the context to answer the question, if not, say I don't know, then we also have context and question which are being passed into this similarly to paramaters in a function."""

'\nAnswer the question based on the context below,                 }\nif you cannot answer the question using the                     }--->  (Rules) For Our Prompt\nprovided information answer with "I don\'t know"                 }\n\nContext: Aurelio AI is an AI development studio                 }\nfocused on the fields of Natural Language Processing (NLP)      }\nand information retrieval using modern tooling                  }--->   Context AI has\nsuch as Large Language Models (LLMs),                           }\nvector databases, and LangChain.                                }\n\nQuestion: Does Aurelio AI do anything related to LangChain?     }--->   User Question\n\nAnswer:                                                         }--->   AI Answer\nHere we can see how the AI will appoach our question, as you can see we have a formulated response, if the context has the answer, then use the context to answer the question, if not, say I don\'t know, then we also have context and qu

In [4]:
prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Context: {context}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "{query}")
]) 

In [6]:
prompt_template.input_variables

['context', 'query']

In [7]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

In [8]:
from langchain.prompts import (
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt),
    HumanMessagePromptTemplate.from_template("{query}")
])

In [9]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

In [10]:
from langchain_groq import ChatGroq

model = "llama3-8b-8192"

llm = ChatGroq(model = model, temperature = 0.0)

In [11]:
context = """Liverpool FC is one of England's most successful football clubs. 
In January 2024, Jurgen Klopp announced he would leave Liverpool at the 
end of the 2023/24 season after nearly nine years as manager. The club 
subsequently appointed Arne Slot from Feyenoord as their new manager 
starting from the 2024/25 season.

The club recently won the 2024 Carabao Cup (League Cup) beating Chelsea 
in the final at Wembley, which was Klopp's final trophy with the club. 
During the 2023/24 season, Liverpool also competed in the Premier League, 
Europa League, and FA Cup, showing strong performances across multiple 
competitions.

The team's current captain is Virgil van Dijk, who took over the 
captaincy after Jordan Henderson's departure to Al-Ettifaq in Saudi 
Arabia during the 2023 summer transfer window. Mohamed Salah remains 
one of the team's key players, despite strong interest from Saudi 
Arabian clubs in 2023."""

query = "Who is Liverpool's current manager and when was the last trophy they won?"

In [12]:
# pipeline.invoke({"query": query, "context" : context})

pipeline = (
    {
        "query": lambda x: x["query"],
        "context" : lambda x : x["context"]
    }
    | prompt_template
    | llm
)

In [13]:
pipeline.invoke({"query": query, "context" : context})

AIMessage(content="According to the context, Liverpool's current manager is Arne Slot, who will start from the 2024/25 season. As for the last trophy they won, it was the 2024 Carabao Cup (League Cup), which they won under the management of Jurgen Klopp.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 282, 'total_tokens': 342, 'completion_time': 0.048796783, 'prompt_time': 0.057755403, 'queue_time': 0.459352908, 'total_time': 0.106552186}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_5b339000ab', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--8d98bb46-f80d-493f-870a-7f76d648a792-0', usage_metadata={'input_tokens': 282, 'output_tokens': 60, 'total_tokens': 342})

### Few Shot Prompting

Many State-of-the-Art (SotA) LLMs are incredible at instruction following. Meaning that it requires much less effort to get the intended output or behavior from these models than is the case for older LLMs and smaller LLMs.

Before creating an example let's first see how to use LangChain's few shot prompting objects. We will provide multiple examples and we'll feed them in as sequential human and ai messages so we setup the template like this:

In [14]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

In [15]:
examples = [
    {"input": "Here is query #1", "output": "Here is the answer #1"},
    {"input": "Here is query #2", "output": "Here is the answer #2"},
    {"input": "Here is query #3", "output": "Here is the answer #3"},
]

In [16]:
from langchain.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples
)

# here is the formatted prompt.
print(few_shot_prompt.format())

Human: Here is query #1
AI: Here is the answer #1
Human: Here is query #2
AI: Here is the answer #2
Human: Here is query #3
AI: Here is the answer #3


In [17]:
new_system_prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Always answer in markdown format. When doing so please
provide headers, short summaries, follow with bullet
points, then conclude.

Context: {context}
"""

prompt_template.messages[0].prompt.template = new_system_prompt

out = pipeline.invoke({"query": query, "context": context}).content
print(out)

### Liverpool FC Manager and Recent Trophy Wins

#### Current Manager
* Arne Slot, appointed from Feyenoord, will take over as Liverpool's new manager starting from the 2024/25 season.

#### Last Trophy Won
* The 2024 Carabao Cup (League Cup), which Liverpool won by beating Chelsea in the final at Wembley, was Jurgen Klopp's final trophy with the club.


In [18]:
from IPython.display import display, Markdown

display(Markdown(out))

### Liverpool FC Manager and Recent Trophy Wins

#### Current Manager
* Arne Slot, appointed from Feyenoord, will take over as Liverpool's new manager starting from the 2024/25 season.

#### Last Trophy Won
* The 2024 Carabao Cup (League Cup), which Liverpool won by beating Chelsea in the final at Wembley, was Jurgen Klopp's final trophy with the club.

In [19]:
"""Answer the user's query based on the context below,                 }
if you cannot answer the question using the                         }
provided information answer with "I don't know"                     }
                                                                    }--->  (Rules)
Always answer in markdown format. When doing so please              }
provide headers, short summaries, follow with bullet                }
points, then conclude. Here are some examples:                      }


User: Can you explain gravity?                                      }
AI: ## Gravity                                                      }
                                                                    }
Gravity is one of the fundamental forces in the universe.           }
                                                                    }
### Discovery                                                       }--->  (Example 1)
                                                                    }
* Gravity was first discovered by...                                }
                                                                    }
**To conclude**, Gravity is a fascinating topic and has been...     }
                                                                    }

User: What is the capital of France?                                }
AI: ## France                                                       }
                                                                    }
The capital of France is Paris.                                     }
                                                                    }--->  (Example 2)
### Origins                                                         }
                                                                    }
* The name Paris comes from the...                                  }
                                                                    }
**To conclude**, Paris is highly regarded as one of the...          }

Context: {context}                                                  }--->  (Context)"""

'Answer the user\'s query based on the context below,                 }\nif you cannot answer the question using the                         }\nprovided information answer with "I don\'t know"                     }\n                                                                    }--->  (Rules)\nAlways answer in markdown format. When doing so please              }\nprovide headers, short summaries, follow with bullet                }\npoints, then conclude. Here are some examples:                      }\n\n\nUser: Can you explain gravity?                                      }\nAI: ## Gravity                                                      }\n                                                                    }\nGravity is one of the fundamental forces in the universe.           }\n                                                                    }\n### Discovery                                                       }--->  (Example 1)\n                                        

In [20]:

examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]

In [21]:
few_shot_prompt2 = FewShotChatMessagePromptTemplate(
    
    example_prompt = example_prompt,
    examples = examples
)

In [22]:
out = few_shot_prompt2.format()

display(Markdown(out))

Human: Can you explain gravity?
AI: ## Gravity

Gravity is one of the fundamental forces in the universe.

### Discovery

* Gravity was first discovered by Sir Isaac Newton in the late 17th century.
* It was said that Newton theorized about gravity after seeing an apple fall from a tree.

### In General Relativity

* Gravity is described as the curvature of spacetime.
* The more massive an object is, the more it curves spacetime.
* This curvature is what causes objects to fall towards each other.

### Gravitons

* Gravitons are hypothetical particles that mediate the force of gravity.
* They have not yet been detected.

**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.


Human: What is the capital of France?
AI: ## France

The capital of France is Paris.

### Origins

* The name Paris comes from the Latin word "Parisini" which referred to a Celtic people living in the area.
* The Romans named the city Lutetia, which means "the place where the river turns".
* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.

**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.



In [23]:
prompt_template2 = ChatPromptTemplate.from_messages([
    ("system", new_system_prompt),
    few_shot_prompt2,
    ("user", "{query}")
])

In [24]:
pipeline2 = prompt_template2 | llm

out2 = pipeline2.invoke({"query": query, "context": context}).content
display(Markdown(out2))

## Liverpool FC

### Current Manager

* Liverpool's current manager is Arne Slot, who will take over from the start of the 2024/25 season.

### Last Trophy Won

* Liverpool won the 2024 Carabao Cup (League Cup) under the management of Jurgen Klopp, who announced his departure from the club at the end of the 2023/24 season.
* This was Klopp's final trophy with the club.

**To conclude**, Liverpool has a rich history of success, and their recent Carabao Cup win is a testament to their continued competitiveness in the football world.

### Chain of Thought Prompting
We'll take a look at one more commonly used prompting technique called chain of thought (CoT). CoT is a technique that encourages the LLM to think through the problem step by step before providing an answer. The idea being that by breaking down the problem into smaller steps, the LLM is more likely to arrive at the correct answer and we are less likely to see hallucinations.

To implement CoT we don't need any specific LangChain objects, instead we are simply modifying how we instruct our LLM within the system prompt. We will ask the LLM to list the problems that need to be solved, to solve each problem individually, and then to arrive at the final answer.

Let's first test our LLM without CoT prompting.

In [25]:
no_cot_system_prompt = """

Be a helpful assistant and answer the user's questions.

You MUST answer the question directly without any other 
text or explanation"""

no_cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", no_cot_system_prompt),
    ("user", "{query}")
])

In [26]:
query = (
    "How many keystrokes are needed to type the numbers from 1 to 500?"
)

no_cot_pipeline = no_cot_prompt_template | llm

no_cot_result = no_cot_pipeline.invoke({"query": query}).content
print(no_cot_result)

2500


In [27]:
# Define the chain-of-thought prompt template
cot_system_prompt = """
Be a helpful assistant and answer the user's question.

To answer the question, you must:

- List systematically and in precise detail all
  subproblems that need to be solved to answer the
  question.

- Solve each sub problem INDIVIDUALLY and in sequence.

- Finally, use everything you have worked through to
  provide the final answer.
"""

In [28]:
cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system" , cot_system_prompt),
    ("user", "{query}" )
])

cot_pipeline = cot_prompt_template | llm

In [29]:
cot_result = cot_pipeline.invoke({"query" : query}).content
display(Markdown(cot_result))

To answer this question, I'll break it down into smaller subproblems and solve each one individually.

Subproblem 1: How many keystrokes are needed to type the numbers from 1 to 9?

* The numbers from 1 to 9 are: 1, 2, 3, 4, 5, 6, 7, 8, 9
* Each number requires 3 keystrokes (1 digit for the number and 2 digits for the decimal point)
* Therefore, the total number of keystrokes needed to type the numbers from 1 to 9 is: 9 x 3 = 27

Subproblem 2: How many keystrokes are needed to type the numbers from 10 to 99?

* The numbers from 10 to 99 are: 10, 11, 12, ..., 98, 99
* Each number requires 4 keystrokes (1 digit for the tens place, 1 digit for the ones place, and 2 digits for the decimal point)
* Since there are 90 numbers in this range (10 to 99), the total number of keystrokes needed is: 90 x 4 = 360

Subproblem 3: How many keystrokes are needed to type the numbers from 100 to 499?

* The numbers from 100 to 499 are: 100, 101, 102, ..., 498, 499
* Each number requires 5 keystrokes (1 digit for the hundreds place, 1 digit for the tens place, 1 digit for the ones place, and 2 digits for the decimal point)
* Since there are 400 numbers in this range (100 to 499), the total number of keystrokes needed is: 400 x 5 = 2000

Subproblem 4: How many keystrokes are needed to type the number 500?

* The number 500 requires 3 keystrokes (1 digit for the hundreds place, 1 digit for the tens place, and 1 digit for the ones place)

Now, let's add up the total number of keystrokes needed to type the numbers from 1 to 500:

* 27 keystrokes for the numbers from 1 to 9
* 360 keystrokes for the numbers from 10 to 99
* 2000 keystrokes for the numbers from 100 to 499
* 3 keystrokes for the number 500

Total number of keystrokes needed to type the numbers from 1 to 500: 27 + 360 + 2000 + 3 = 2390

Therefore, it takes 2390 keystrokes to type the numbers from 1 to 500.

### Finally, as mentioned most LLMs are now trained to use CoT prompting by default. So let's see what happens if we don't explicitly tell the LLM to use CoT.

In [30]:
system_prompt = """
Be a helpful assistant and answer the user's question.
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{query}"),
])

pipeline = prompt_template | llm

In [31]:
result = pipeline.invoke({"query": query}).content
display(Markdown(result))

What a great question!

To calculate the number of keystrokes needed to type the numbers from 1 to 500, we need to consider the following:

* Each digit (0-9) requires 1 keystroke.
* The number of digits in each number increases as we go from 1 to 500.

Here's a breakdown:

* Numbers from 1 to 9 require 1 digit each, so 9 keystrokes.
* Numbers from 10 to 99 require 2 digits each, so 90 keystrokes (9 x 2).
* Numbers from 100 to 499 require 3 digits each, so 400 keystrokes (4 x 100).
* The number 500 requires 3 digits, so 3 keystrokes.

Adding all these up, we get:

9 + 90 + 400 + 3 = 502 keystrokes

So, it takes a total of 502 keystrokes to type the numbers from 1 to 500.